## Source: https://www.compare-school-performance.service.gov.uk/schools-by-type?step=default&table=schools&region=936&geographic=la&for=primary

In [ ]:
import pandas as pd
import glob

In [ ]:
# Path of school folder
path = 'school'

In [ ]:
# Read all files in folder
files = glob.glob(path + '\*.csv')
files_dict = {}

for file in files:
    df = pd.read_csv(file)
    files_dict[file.split('_')[1].split('.')[0]] = df

In [ ]:
# Read Elmbridge Postal Codes
postal_codes_df = pd.read_csv('Elmbridge Postcodes.csv')
postal_codes_df_clean = postal_codes_df[['Postcode', 'Latitude', 'Longitude']]

In [ ]:
# Clean ks4-pupdest
df_ks4_pupdest = files_dict['ks4-pupdest']

In [ ]:
# Clean school
df_school = files_dict['school']

# Clean columns
df_school_clean = (df_school[['URN', 'STREET', 'TOWN', 'POSTCODE', 'MINORGROUP', 'SCHOOLTYPE', 'ISPRIMARY', 
                             'ISSECONDARY', 'ISPOST16', 'GENDER', 'OFSTEDRATING']]).copy()
df_school_clean.columns = ['ID', 'Street', 'Town', 'Postcode', 'Type 1', 'Type 2', 'Primary', 'Secondary',
                           'Post 16', 'Gender', 'Rating']

# Get Postal Codes
df_school_clean['Postal Code 1'] = (df_school_clean['Postcode'].str.split(' ').str[0])
df_school_clean['Postal Code 2'] = (df_school_clean['Postcode'].str.split(' ').str[1])

# Filter by Postal Codes
df_school_merged = df_school_clean.merge(postal_codes_df_clean, how = 'left', on = 'Postcode')
df_school_merged.dropna(subset = ['Latitude'], inplace = True)
df_school_merged.reset_index(drop = True)

In [ ]:
# Clean swf
df_swf = files_dict['swf']

# Clean columns
df_swf_clean = df_swf[['URN', 'School Phase', 'Total Number of Teachers (Headcount)', 
                 'Total Number of Non Classroom-based School Support Staff, Excluding Auxiliary Staff (Headcount)',
                 'Total Number of Teachers (Full-Time Equivalent)', 
                 'Total Number of Teaching Assistants (Full-time Equivalent)', 
                 'Total Number of Non Classroom-based School Support Staff, Excluding Auxiliary Staff (Full-Time Equivalent)',
                 'Pupil:     Teacher Ratio', 'Mean Gross FTE Salary of All Teachers (£s)']]

df_swf_clean.columns = ['ID', 'School Phase', 'Number of Teachers', 'Number of TAs', 'Number of Support Staff', 
                 'Number of Full-Time TAs', 'Number of Full-Time Support Staff', 'Student:Teacher Ratio', 
                 'Mean Salary of All Teachers']

# Drop null URN
df_swf_clean = df_swf_clean.dropna(subset = 'ID')

# Merge other school info table
df_school_merged = df_school_merged.merge(df_swf_clean, how = 'left', on = 'ID')
df_school_merged.to_excel('schools clean.xlsx')